In [37]:
for set in ['test', 'acl']:
    for i in range(3):
        with open(f'generations/get_generations/{set}_prompts_{i}.py', 'w') as f:
            f.write(f"""import google.generativeai as genai
import pandas as pd
from tqdm import tqdm
import re

tqdm.pandas()
partition_num = '{i}'
df = pd.read_csv(f'../generation_prompts/{set}_chunk_{i}.csv')

genai.configure(api_key="{api_keys[i%3]}")
model = genai.GenerativeModel('gemini-pro')

def get_response(row):
    try:
        response = model.generate_content(row.Prompt)
    except Exception as e:
        print(e)
        return
    try:
        return response.text
    except Exception as e:
        try:
            pattern = r'\*\*Response:\*\*(.*?)\\n\\n\*\*Additional Information:\*\*'
            return re.search(pattern, r.parts[0].text, re.DOTALL).group(1).strip()
        except Exception as x:
            print(e)
            print('***AND***')
            print(x)

df['response'] = df.progress_apply(get_response, axis=1)
df.to_csv(f'../generation_responses/{set}_chunk_{i}.csv', index=False)
""")

In [29]:
import google.generativeai as genai
import time
import re

In [13]:
genai.configure(api_key=f"{api_keys[0]}")
model = genai.GenerativeModel('gemini-pro')

def get_response(row):
    try:
        start_time = time.time()
        response = model.generate_content(row)
    except Exception as e:
        print(e)
    try:
        elapsed_time = time.time()-start_time
        if elapsed_time < 6:
            time.sleep(6-elapsed_time)
        if response:
            return response
    except Exception as e:
        print(e)

In [14]:
r = get_response('testing 123')

In [15]:
r

In [28]:
r.parts[0].text.split('\n\n')

['**Assistant**',
 '**Input:** testing 123',
 "**Response:** Hello! I'm here to help you with your testing needs. How can I assist you today?",
 "**Additional Information:**\nThis response demonstrates the assistant's ability to acknowledge the user's input and express its readiness to assist with testing-related tasks. It provides a friendly and helpful tone, encouraging the user to initiate a conversation. This response can be adapted to various contexts, such as software testing, web testing, or any other type of testing activity."]

In [33]:
re.search(r'\*\*Response:\*\*(.*?)\n\n\*\*Additional Information:\*\*', r.parts[0].text, re.DOTALL).group(1).strip()

"Hello! I'm here to help you with your testing needs. How can I assist you today?"